In [1]:
import fxcmpy
import socketio
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sympy import *
init_printing()
import xlrd
import itertools
from datetime import datetime as dt
import pytz
from itertools import combinations
from itertools import product
from itertools import groupby
from astropy.table import QTable, Table, Column
from astropy import units as u
from IPython.display import HTML
import requests
import json
from scipy.signal import argrelextrema

TOKEN = 'b199bb5a15fa26957990a3fb823a51bd473cca55'
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error', server='demo', log_file='log.txt')

C:\ProgramData\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [10]:
##  INITIAL SETUP

## WATCHLIST = ['AUD/CAD','AUD/CHF','AUD/JPY','AUD/NZD','AUD/USD','CAD/CHF','CAD/JPY','CHF/JPY','EUR/AUD','EUR/CAD','EUR/CHF','EUR/GBP','EUR/JPY','EUR/NZD','EUR/USD','GBP/AUD','GBP/CAD','GBP/CHF','GBP/JPY','GBP/USD','NZD/CAD','NZD/CHF','NZD/JPY','NZD/USD','USD/CAD','USD/CHF','USD/JPY']

pair = ['EUR/JPY']
h = len(pair)


with pd.ExcelWriter('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_I/MarkI_Levels.xlsx') as writerdft:
    with pd.ExcelWriter('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_I/MarkI_Results.xlsx') as writerdfd:

        for x in pair:
            
## TO BEGIN, WE WILL RUN A SHORT PROGRAM TO FIGURE OUT HOW TO SET THE TIMEFRAME FOR THE LEVELS CALCULATION


            Q = con.get_candles(x, period='H4', number=120)      ## START WITH 120 CANDLESTICKS (20 DAYS) FOR CURRENT MIDPOINT
            Q['open'] = (Q['bidopen']+Q['askopen'])/2
            Q['close'] = (Q['bidclose']+Q['askclose'])/2
            Q['high'] = (Q['bidhigh']+Q['askhigh'])/2
            Q['low'] = (Q['bidlow']+Q['asklow'])/2
            Qmid = (np.max(Q['high'])+np.min(Q['low']))/2
            Qrange = (np.max(Q['high'])-Qmid)

            W = con.get_candles(x, period='H4', number=2880)    ## NOW MOVE TO 2880 CANDLESTICKS (2 YEARS) FOR SKEWING
            W['open'] = (W['bidopen']+W['askopen'])/2
            W['close'] = (W['bidclose']+W['askclose'])/2
            W['high'] = (W['bidhigh']+W['askhigh'])/2
            W['low'] = (W['bidlow']+W['asklow'])/2
            wlen = W['close'].count()
            wlin = np.linspace(1,wlen,num=wlen)
            Windex = W.index

            RangeDiff = []
            i = 0
            for t in wlin:
                wdiff = abs(Qmid - ((W.iloc[i,9]+W.iloc[i,10])/2))
                i = i + 1
                RangeDiff.append(wdiff)

            RangeDiff = [x <= Qrange for x in RangeDiff]
            res = next(x for x, val in enumerate(RangeDiff) 
                                          if val == 1) 

            startdate = Windex[res]
            startdate = startdate.strftime('%Y %m %d %H:%M:%S')
            today = dt.today()
            

## NOW, WE HAVE A TIME INTERVAL SET FOR LEVELS AND CAN START CALCULATING WHERE THEY ARE


            S = con.get_candles(x, period='H4', start=startdate, end=today) 
            S = S.tz_localize('Etc/GMT-4')
            S = S.tz_convert('UTC')
            S['open'] = (S['bidopen']+S['askopen'])/2
            S['close'] = (S['bidclose']+S['askclose'])/2
            S['high'] = (S['bidhigh']+S['askhigh'])/2
            S['low'] = (S['bidlow']+S['asklow'])/2

            S = S.assign(MOVE=(S['close'] - S['open']))
            AvgCandle = np.average(abs(S['MOVE']))
            CP = S['close'].iloc[-1]
            l = int(S['close'].count())

            CPU = CP + AvgCandle*20                        ## THIS IS THE % MARGIN FOR THE SCOPE OR # OF LEVELS
            CPD = CP - AvgCandle*20

            CM = AvgCandle*1                                 ## THIS IS THE % MARGIN FOR THE CONFLUENCE OF LEVELS

            CNDL = (S.iloc[:, 10]) - (S.iloc[:, 9])
            S['CNDL'] = np.where(CNDL > 0, 1, -1)
            S = S.assign(INFL=S['CNDL'].ne(S['CNDL'].shift()))
            S = S[S.INFL != 0]
            S['INFL'] = S['CNDL']*-1
            S['LVL'] = np.where(S.INFL == 1, S.high, S.low)
            S = S.assign(SIZE=abs(S['LVL'] - (S['LVL'].shift(-1))))

            A = np.array(S.LVL)

            B = list(combinations(A, 2))
            Ba = np.array([x for x, y in B])                       #### HERE'S WHERE TO ROUND THE LEVEL GROUPINGS
            B = np.array([abs(x - y) for x, y in B])
            Bb = np.array([np.where(x <= CM, 1, 0) for x in B])             #### RULE WHERE CONFLUENCE MARGIN COMES INTO PLAY

            T = list(zip(Ba,Bb))

            table = pd.DataFrame(T, columns=['a','b']).groupby('a').sum()
            table['max'] = table.b[(table.b.shift(1) < table.b) & (table.b.shift(-1) < table.b)]
            table = table.sort_values(['a'], ascending=False)
            table = table.rename(columns={'b': 'Pings'})
            table = table[table.index <= CPU]
            table = table[table.index >= CPD]
            table = table.fillna(0)
            table = table[table['max'] != 0]

            crank1 = 1                                                  ## CRANK 1 TO ADJUST LEVEL ACCURACY
            tabmean = np.average(table['max'])
            tabstd = np.std(table['max'])
            tablim = tabstd*crank1 + tabmean
            table = table[table['max'] > tablim]

            crank2 = 1                                             ## CRANK 2 TO ADJUST LEVEL ACCURACY
            table = table.assign(SPACE=(table.index))
            table.SPACE = abs(table.SPACE - table.SPACE.shift())      
            spacemean = np.average(table.iloc[1:,2])
            spacestd = np.std(table.iloc[1:,2])
            spacelim = spacestd*crank2 + spacemean
            table = table[table['SPACE'] >= spacelim]
            
            levels = table.index


In [11]:
print(levels)

Float64Index([117.32249999999999], dtype='float64', name='a')
